# 에지 검출과 응용
* 에지 검출
* 직선 검출과 원 검출

# 에지 검출

## 미분과 그래디언트
* 에지 검출? ==> 픽셀 값의 **변화율**이 큰 픽셀! (급격한 밝기 변화)
> **(순간)변화율 == 미분(derivative)**
* f'(x) 값이 0보다 훨씬 크거나 훨씬 작은 위치 ==> 급격한 밝기 값 변화 ==> 에지!

![](f'(x).png)  
여기서 a, b, c 지점에서 각각 에지 가능성 높음

**영상으로부터 미분 계산 시 고려할 사항 2가지**
* 영상은, 2차원 평면에서 정의된 **함수**이다. (영상 == f(x))
* 영상은, 정수 단위 좌표 위치에 픽셀이 나열되어 있는 **이산함수**이다.  
> 따라서, **미분 근사화 방법** 이용해서 변화량 측정!  
(미분 근사화 방법 : 1차원 이산함수에서 미분 구하는 방법)

## 미분 근사
h = 1 (픽셀 간격의 최소 단위)
* 전진 차분(forward difference) : ( I(x+h) - I(x) )  / h
* 후진 차분(backward difference) : ( I(x) - I(x-h) )  / h
* **중앙 차분(centered difference) : ( I(x+h) - I(x-h) )  / 2h**
> 중앙 차분 방법이 가장 good

## 2차원 평면에서의 미분
* 영상을 가로방향(Ix), 세로방향(Iy) **각각 편미분** 해줘야 함  
![](2차원미분.png)
* 미분 값의 상대적 크기 비교만 하면 되므로, 1/2는 해주지 않고 마스크 사용 (-1, 0, 1)
* 미분 결과 영상에서,
    * 흰색 부분 : 픽셀 값이 급격하게 증가하는 부분
    * 검은색 부분 : 픽셀 값이 급격하게 감소하는 부분

## 그래디언트 (gradient)
* f(x, y)의 x축 방향 미분과 y축 방향 미분을 **한꺼번에 벡터로** 표현한 것
    * 그래디언트 방향(phase) : 변화 정도가 가장 큰 방향 (*밝기 가장 밝아지는 방향*)
    * 그래디언트 크기(magnitude) : 변화율 세기 ==> **|| df ||**  
    
> 밝기 차이가 클수록(c) 그래디언트 벡터의 크기 커짐! (더 긴 화살표)  
![](그래디언트.png)  
여기서 **빨간색** 화살표 : **그래디언트 벡터** / **노란색** 화살표 : **에지!** (그래디언트와 수직)  
* ***그래디언트 벡터(빨간색)*** : fx(x축 변화율)와 fy(y축 변화율)를 통해서 구할 수 있음  
* cf. ***c점에서, f(x+1) - f(x-1) < 0 이므로 왼쪽(서쪽)으로 fx 나타남***

* 에지의 방향 ==> 그래디언트 벡터와 수직 방향 (크기는 같음, 즉 변화율 세기로 에지 검출)

## 마스크 기반 에지 검출
* 3x3 소벨 마스크 : 마스크 요소 합 = 0
* 소벨 마스크로 현재 행뿐만 아니라 **이웃 행에서도** 마스크 연산 하는 이유?
    * **잡음의 영향을 줄이기 위해!** (현재 행 & 이웃 행에서의 픽셀 값 변화가 유사하므로)
    > * 가운데(현재) 행 : [-2, 0, 2]와 같이 중앙 차분 연산(-1, 0, 1) 두 번씩!
    > * 양 옆 이웃 행들 : [-1, 0, 1] 한 번씩 연산 수행하여, 주변부도 같이 고려!

## (실습) 마스크 기반 에지 검출

In [ ]:
import cv2
import numpy as np

def sobel_derivative():
    src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)
    
    # x방향, y방향 마스크 필터 정의
    mx = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]], dtype=np.float32)
    my = np.array([[-1, -2, -1],
                   [0, 0, 0],
                   [1, 2, 1]], dtype=np.float32)
    
    # 소벨 마스크로 미분 연산 수행 (= 마스크(컨볼루션) 연산 = 필터링 = 에지 검출!)
    dx = cv2.filter2D(src, -1, mx, delta=128) # 128 : 값이 음수로 튀는 경우 방지
    dy = cv2.filter2D(src, -1, my, delta=128)
    
    cv2.imshow('src', src)
    cv2.imshow('dx', dx)
    cv2.imshow('dy', dy)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
sobel_derivative()

![](mask_edge_result.png)

## 소벨(Sobel) 필터
* OpenCV에서 제공하는 소벨 마스크를 이용한 영상 미분 함수
* **dst = cv2.Sobel(src, ddepth, dx, dy)**
    * dst : src를 편미분한 결과
    * ddepth : dst 영상의 **자료형** (**-1** : src와 같은 타입의 영상 생성)
    * dx, dy : x방향, y방향으로의 편미분 차수 (대부분 1차 미분)